In [3]:
import xlwings as xw
import re
import pandas as pd

# pd.options.mode.copy_on_write = True

In [4]:
# Excel File 읽기
xw_bom = xw.Book(r'example/bom.xlsx')
xw_cost = xw.Book(r'example/cost.xlsx')
xw_inventory = xw.Book(r'example/inventory.xlsx')

In [5]:
# 특정 sheet 읽기
sh_bom = xw_bom.sheets(1)
sh_cost = xw_cost.sheets(1)
sh_inventory = xw_inventory.sheets(1)

In [6]:
# Dataframe으로 변환
df_bomlist = sh_bom.range('A1').options(pd.DataFrame, index=False, expand='table').value
df_cost = sh_cost.range('A1').options(pd.DataFrame, index=False, expand='table').value
df_inventory = sh_inventory.range('A1').options(pd.DataFrame, index=False, expand='table').value

In [7]:
#공백제거
df_bomlist['Part No.'] = df_bomlist['Part No.'].str.replace(" ","")
df_cost['품명'] = df_cost['품명'].str.replace(" ","")
df_inventory['자재명'] = df_inventory['자재명'].str.replace(" ","")

In [37]:
# BOM에서 필요한 Column만 추출 및 생성
df_partname = df_bomlist["Part No."].dropna()
df_qty = df_bomlist["Q'TY"].dropna().astype(int)
df_bom = pd.concat([df_partname,df_qty], ignore_index=True, axis=1)
df_bom = df_bom.reset_index(drop=True)
df_bom = df_bom.rename(columns = {0:'품명', 1:'사용수량'})
df_bom['단가'] = [0 for i in range(len(df_bom))]
df_bom['현재고'] = [0 for i in range(len(df_bom))]
df_bom['출고제한수량'] = [0 for i in range(len(df_bom))]
df_bom['프로젝트번호'] = [0 for i in range(len(df_bom))]
# df_partname
# df_qty
df_bom

,품명,사용수량,단가,현재고,출고제한수량,프로젝트번호
0,C0603C105J4RACAUTO,114,0,0,0,0
1,C0603C104J5RACAUTO,91,0,0,0,0
2,C0603C103J1RACAUTO,87,0,0,0,0
3,T495A106K010AHA2K0,13,0,0,0,0
4,VRF-04-10-50-04-N,2,0,0,0,0
5,VSF-05-20-50-04-G,1,0,0,0,0
6,HIBISCUS,1,0,0,0,0
7,T495A106K010AHA2K0,29,0,0,0,0
8,T495B686K010AHE600,4,0,0,0,0
9,C0805C105J4RACAUTO,1,0,0,0,0


In [38]:
# 단가 추가
for k in range(len(df_bom)):
    datafilter = df_cost['품명'].notnull() & df_cost['품명'].str.contains(str(df_bom.iloc[k,0])[:],case=False) # 대소문자 구분X
    if len(df_cost.loc[datafilter,'최종결산월재고단가'].values) == 0:
        df_bom['단가'][k] = 0
    elif len(df_cost.loc[datafilter,'최종결산월재고단가'].values) > 1 : # 동일한 부품단가가 있는 경우 마지막값 사용
        df_bom['단가'][k] = int(df_cost.loc[datafilter,'최종결산월재고단가'].values[-1])
    else:
        df_bom['단가'][k] = int(df_cost.loc[datafilter,'최종결산월재고단가'].values)
df_bom

C:\Users\sangi\AppData\Local\Temp\ipykernel_3728\3174930533.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_bom['단가'][k] = 0
C:\Users\sangi\AppData\Local\Temp\ipykernel_3728\3174930533.py:5: SettingWithCopyWarning: 
A value is trying 

,품명,사용수량,단가,현재고,출고제한수량,프로젝트번호
0,C0603C105J4RACAUTO,114,0,0,0,0
1,C0603C104J5RACAUTO,91,0,0,0,0
2,C0603C103J1RACAUTO,87,0,0,0,0
3,T495A106K010AHA2K0,13,0,0,0,0
4,VRF-04-10-50-04-N,2,114757,0,0,0
5,VSF-05-20-50-04-G,1,99104,0,0,0
6,HIBISCUS,1,0,0,0,0
7,T495A106K010AHA2K0,29,0,0,0,0
8,T495B686K010AHE600,4,0,0,0,0
9,C0805C105J4RACAUTO,1,0,0,0,0


In [41]:
# 재고 추가
for k in range(len(df_bom)):
    datafilter = df_inventory['자재명'].notnull() & df_inventory['자재명'].str.contains(str(df_bom.iloc[k,0])[:],case=False) # 대소문자 구분X
    # print(df_inventory[datafilter]['프로젝트번호'].values)
    if len(df_inventory[datafilter]['프로젝트번호'].values) == 0:
        pass
        # df_bom['현재고'][k] = 0
        # df_bom['출고제한수량'][k] = str()
        # df_bom['프로젝트번호'][k] = str()
    else:
        df_bom['현재고'][k] = str(df_inventory[datafilter]['현재고'].values)
        df_bom['출고제한수량'][k] = str(df_inventory[datafilter]['출고제한수량'].values)
        df_bom['프로젝트번호'][k] = str(df_inventory[datafilter]['프로젝트번호'].values)

df_bom

C:\Users\sangi\AppData\Local\Temp\ipykernel_3728\701314064.py:11: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_bom['현재고'][k] = str(df_inventory[datafilter]['현재고'].values)
C:\Users\sangi\AppData\Local\Temp\ipykernel_3728\701314064.py:11: 

,품명,사용수량,단가,현재고,출고제한수량,프로젝트번호
0,C0603C105J4RACAUTO,114,0,0,0,0
1,C0603C104J5RACAUTO,91,0,0,0,0
2,C0603C103J1RACAUTO,87,0,0,0,0
3,T495A106K010AHA2K0,13,0,0,0,0
4,VRF-04-10-50-04-N,2,114757,[4. 8. 2.],[0. 8. 2.],['G-C4P20B' 'A-STP21A' 'G-MSC20B']
5,VSF-05-20-50-04-G,1,99104,[7. 2. 3. 4. 6.],[0. 0. 3. 4. 5.],['N-PIO21A' 'G-C4P20B' 'A-STP21A' 'A-STP21A' '...
6,HIBISCUS,1,0,0,0,0
7,T495A106K010AHA2K0,29,0,0,0,0
8,T495B686K010AHE600,4,0,0,0,0
9,C0805C105J4RACAUTO,1,0,0,0,0


In [42]:
# CSV 출력
df_bom.to_csv("result.csv",encoding='cp949')